# 게임에서 이탈하는 사람들은 아이템을 정리하는 경향을 보이는가?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
label = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/2019빅콘테스트_챔피언스리그_데이터_수정/train_label.csv')
payment = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/2019빅콘테스트_챔피언스리그_데이터_수정/train_payment.csv')
act = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/2019빅콘테스트_챔피언스리그_데이터_수정/train_activity.csv')
trade = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/2019빅콘테스트_챔피언스리그_데이터_수정/train_trade.csv')



In [37]:
trade = pd.read_csv('C:/Users/CPB06GameN/Desktop/PROGRAM/big/2019빅콘테스트_챔피언스리그_데이터_수정/train_trade.csv')


print(trade.shape, trade[trade['type']==1].shape, trade[trade['type']==0].shape)\

private_shop = trade[trade['type']==0]
trade_shop = trade[trade['type']==1]


len(set(trade.source_acc_id))
# len(set(label.acc_id))
trade1 = trade.rename(columns = {'source_acc_id':'acc_id'})
trade2 = trade.rename(columns = {'acc_id':'source_acc_id',
                                 'target_acc_id':'acc_id'})
trade1.head()

(1849325, 11) (1040143, 11) (809182, 11)


,day,time,type,server,acc_id,source_char_id,target_acc_id,target_char_id,item_type,item_amount,item_price
0,7,21:13:05,1,ag,11439,385109,48152,34247,enchant_scroll,4.793968e-08,NaN
1,2,15:33:00,1,al,22220,259705,43228,379826,enchant_scroll,4.314571e-06,NaN
2,1,14:24:53,1,al,18212,164230,38145,34209,accessory,7.190952e-08,NaN
3,7,23:13:43,1,au,121622,49839,71226,281334,adena,9.587936e-03,NaN
4,2,12:43:33,0,bd,14421,344503,65220,210558,etc,1.198492e-07,0.000639


## LABEL에 있는 인원들 중 TRADE에도 있는 사람들

중에 나는 판매자가 게임을 그만두는지를 알고 싶기 때문에 

판매자인 trade_label_sell 로 선택하여 분석해보려고 한다.


> trade_label_sell 에는 label에 있는 40000명중 28874만 존재한다.


In [39]:
trade_label_sell = label.merge(trade1, on='acc_id', how='inner')
print(trade_label_sell.shape, len(set(trade_label_sell.acc_id)) )

trade_label_buy = label.merge(trade2, on='acc_id', how='inner')
print(trade_label_buy.shape, len(set(trade_label_buy.acc_id)) )

(1403720, 13) 28874
(1280083, 13) 31373


그리고 그 28874명 중 

17899명은 64일까지 잔존한 인원들이고,

10975명은 중간에 이탈한 인원이다.

In [62]:
sell = trade_label_sell.groupby(['acc_id'], as_index=False).mean()
print(sell.shape[0])

print(sell[sell['survival_time']==64].shape[0], sell[sell['survival_time']!=64].shape[0])


28874
17899 10975


label 데이터에서 실제로 그만 둔 인원은 18004명이고,

그 인원들 중 

10975명이 아이템을 판매한 인원들이며, 

1012명이 1일차에 그만둔 사람들이다.

In [53]:
label[label['survival_time']!=64].shape[0], label[label['survival_time']==1].shape[0]

(18004, 1012)

# 21일 이후의 거래 데이터 & 7일 이하 생존자들

단순히 28일치의 거래 데이터로 64일을 모두 예측하는 것은 생각했던 것 만큼의 설명력이 없다고 판단..!

때문에 그래도 설명할 수 있다고 판단되는 21일 이후의 거래 데이터 & 7일 이하 생존자들을 살펴보기로 한다.

In [69]:
label[label['survival_time']<8].shape[0]
label_accid = label[label['survival_time']<8]['acc_id']
label_accid = set(label_accid)
len(label_accid)

3721

7일 이전에 그만둔 인원은 3721명.

In [71]:
sell1 = trade_label_sell.groupby(['day','acc_id'], as_index=False).count()
sell2 = sell1[['day','acc_id','item_type']]
 
sell_21 = sell2[sell2['day']>20]
sell_acc_id = sell_21[sell_21['item_type']>np.mean(sell_21['item_type'])]['acc_id']
sell_acc_id = set(sell_acc_id)
len(sell_acc_id)

5791

21일 이후에 판매를 많이 하는 인원은 5791명.

두 집단의 유저아이디를 집합으로 만들어서 교집합을 구하는 형태로 얼마나 일치하는지를 파악한다.

In [74]:
len(sell_acc_id.intersection(label_accid))

249

249명... 이 가설은 기각하는 것으로 결론.